In [1]:
import pandas as pd
import matplotlib
import seaborn as sns
import cufflinks as cf
import numpy as np
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

cf.go_offline()
data = pd.read_csv(r'C:\Users\Abhinay\Downloads\UCI_Credit_Card.csv')
pd.pandas.set_option('display.max_columns',None)
data.head()
# No Null or missing Values in DataSet

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [2]:
# Selection of specific features
#selected_features_data = pd.concat([data.iloc[:, 6:12] ,data.iloc[:, 1:2], data.iloc[:, 18:24]], axis=1)
selected_features_data = pd.concat([data.iloc[:, 6:12] ,data.iloc[:, 1:2], data.iloc[:, 18:20]], axis=1)
selected_features_data

,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,LIMIT_BAL,PAY_AMT1,PAY_AMT2
0,2,2,-1,-1,-2,-2,20000.0,0.0,689.0
1,-1,2,0,0,0,2,120000.0,0.0,1000.0
2,0,0,0,0,0,0,90000.0,1518.0,1500.0
3,0,0,0,0,0,0,50000.0,2000.0,2019.0
4,-1,0,-1,0,0,0,50000.0,2000.0,36681.0
...,...,...,...,...,...,...,...,...,...
29995,0,0,0,0,0,0,220000.0,8500.0,20000.0
29996,-1,-1,-1,-1,0,0,150000.0,1837.0,3526.0
29997,4,3,2,-1,0,0,30000.0,0.0,0.0
29998,1,-1,0,0,0,-1,80000.0,85900.0,3409.0


# Splitting dataset and applying RandomUnderSampling

In [3]:
X = selected_features_data
Y = data.iloc[:,-1]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.7)


sampler = RandomUnderSampler(random_state=29, replacement=True)
X_train_resampled, Y_train_resampled = sampler.fit_resample(X_train, Y_train)

In [4]:
X_train_resampled.shape, Y_train_resampled.shape, X_test.shape, Y_test.shape

((9254, 9), (9254,), (9000, 9), (9000,))

# Randomized Hyper-parameter tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import time

model = RandomForestClassifier()
params = {'criterion':['gini','entropy'],
         'min_samples_split': range(2,21),
         'n_estimators': range(50,550,50),
         'max_depth': range(2,26),
          'min_samples_leaf': range(2,11),
          'bootstrap': [True,False]
        }
gscv = RandomizedSearchCV(model, param_distributions=params, cv=10, scoring='f1', n_iter=100)

print(time.ctime(time.time()))
gscv.fit(X_train_resampled, Y_train_resampled)
print(time.ctime(time.time()))


In [ ]:
gscv.best_params_

In [9]:
print(time.ctime(time.time()))

model = RandomForestClassifier(n_estimators=600, max_depth=10, criterion='entropy', bootstrap=False, min_samples_leaf=7, min_samples_split=10)
model.fit(X_train_resampled, Y_train_resampled)

print(time.ctime(time.time()))

Tue Nov 29 01:42:40 2022
Tue Nov 29 01:42:52 2022


In [10]:
pred = model.predict(X_test)
print(model.score(X_train_resampled, Y_train_resampled))
print(accuracy_score(Y_test, pred))
print(classification_report(pred, Y_test))

0.754700669980549
0.7702222222222223
              precision    recall  f1-score   support

           0       0.83      0.87      0.85      6649
           1       0.57      0.49      0.53      2351

    accuracy                           0.77      9000
   macro avg       0.70      0.68      0.69      9000
weighted avg       0.76      0.77      0.76      9000

